<a href="https://colab.research.google.com/github/jayrom/fiap_exercises_gen/blob/main/reply_test/train_data_generation_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulação de dados de sensores industriais
*Dataset de manutenção (rotulado)*

In [2]:
import pandas as pd
import numpy as np

# Seed para reprodutibilidade
np.random.seed(42)

# --- Configurações Iniciais (mantidas do seu script) ---
n_leituras_total = 500
intervalo_minutos = 10
start_time = pd.Timestamp('2025-01-01 00:00:00') # Atualizei o ano para o corrente
timestamps = pd.to_datetime([start_time + pd.Timedelta(minutes=i*intervalo_minutos) for i in range(n_leituras_total)])

# ==============================================================================
# Motor A: Falha tipo rolamento/desgaste
# ==============================================================================
device_id_A = ['motor_A'] * n_leituras_total

# Dados normais para motor A (temperatura e corrente)
temp_A = np.random.normal(30, 2, n_leituras_total)
current_A = np.random.normal(1.5, 0.1, n_leituras_total)

# --- ALTERAÇÃO: Simulação de vibração em 3 eixos para Motor A ---
# Em estado normal, a vibração é baixa em todos os eixos.
# Adicionamos a aceleração da gravidade (aprox. 9.8 m/s^2) ao eixo Z para maior realismo.
vib_x_A = np.random.normal(0, 0.05, n_leituras_total)
vib_y_A = np.random.normal(0, 0.05, n_leituras_total)
vib_z_A = np.random.normal(9.8, 0.05, n_leituras_total)

# Simular falha progressiva para motor A
start_failure_A_idx = int(n_leituras_total * 0.7)
days_to_failure_A = np.ones(n_leituras_total) * -1

for i in range(start_failure_A_idx, n_leituras_total):
    progress = (i - start_failure_A_idx) / (n_leituras_total - start_failure_A_idx)

    # Aumento não linear para temperatura e corrente (lógica mantida)
    temp_A[i] += 15 * np.sin(progress * np.pi) + np.random.normal(0, 1)
    current_A[i] += 5 * np.sin(progress * np.pi) + np.random.normal(0, 0.5)

    # --- ALTERAÇÃO: Falha de rolamento afetando mais os eixos radiais (X e Y) ---
    # O padrão senoidal que você criou é aplicado aqui
    vib_x_A[i] += 1.0 * np.sin(progress * np.pi) + np.random.normal(0, 0.1)
    vib_y_A[i] += 1.2 * np.sin(progress * np.pi) + np.random.normal(0, 0.1) # Um pouco mais forte em Y
    vib_z_A[i] += 0.3 * np.sin(progress * np.pi) + np.random.normal(0, 0.05) # Menor impacto no eixo Z

    days_remaining = (n_leituras_total - 1 - i) * intervalo_minutos / (60 * 24)
    days_to_failure_A[i] = max(0, days_remaining)

# --- NOVO: Calcular a magnitude vetorial a partir dos 3 eixos ---
vibration_magnitude_A = np.sqrt(vib_x_A**2 + vib_y_A**2 + vib_z_A**2)

failure_mode_A = np.where(days_to_failure_A != -1, 'desgaste_rolamento', 'normal')

# ==============================================================================
# Motor B: Falha desbalanceamento/acoplamento
# ==============================================================================
device_id_B = ['motor_B'] * n_leituras_total

# Dados normais para motor B (temperatura e corrente)
temp_B = np.random.normal(32, 1.8, n_leituras_total)
current_B = np.random.normal(1.7, 0.15, n_leituras_total)

# --- ALTERAÇÃO: Simulação de vibração em 3 eixos para Motor B ---
vib_x_B = np.random.normal(0, 0.06, n_leituras_total)
vib_y_B = np.random.normal(0, 0.06, n_leituras_total)
vib_z_B = np.random.normal(9.8, 0.06, n_leituras_total)

# Simular falha progressiva para motor B
start_failure_B_idx = int(n_leituras_total * 0.6)
days_to_failure_B = np.ones(n_leituras_total) * -1

for i in range(start_failure_B_idx, n_leituras_total):
    progress = (i - start_failure_B_idx) / (n_leituras_total - start_failure_B_idx)

    # Temperatura e corrente sobem com delay (lógica mantida)
    if progress > 0.3:
        temp_B[i] += 10 * np.sqrt(progress - 0.3) + np.random.normal(0, 1.2)
        current_B[i] += 4 * np.sqrt(progress - 0.3) + np.random.normal(0, 0.6)

    # --- ALTERAÇÃO: Desbalanceamento criando uma vibração forte e direcional (principalmente no eixo Y) ---
    # O padrão exponencial que você criou é aplicado aqui
    vib_x_B[i] += 0.5 * np.exp(progress * 1.5) + np.random.normal(0, 0.10)
    vib_y_B[i] += 2.5 * np.exp(progress * 1.5) + np.random.normal(0, 0.20) # Aumento exponencial muito mais forte em Y
    vib_z_B[i] += 0.2 * np.exp(progress * 1.5) + np.random.normal(0, 0.08)

    days_remaining = (n_leituras_total - 1 - i) * intervalo_minutos / (60 * 24)
    days_to_failure_B[i] = max(0, days_remaining)

# --- NOVO: Calcular a magnitude vetorial a partir dos 3 eixos ---
vibration_magnitude_B = np.sqrt(vib_x_B**2 + vib_y_B**2 + vib_z_B**2)

failure_mode_B = np.where(days_to_failure_B != -1, 'desbalanceamento', 'normal')

# ==============================================================================
# --- Criação e Consolidação dos DataFrames ---
# ==============================================================================

# Criar DataFrames para cada motor usando a nova coluna de magnitude de vibração
df_motor_A = pd.DataFrame({
    'timestamp': timestamps,
    'device_id': device_id_A,
    'temperature_c': temp_A,
    'current_amps': current_A,
    'vibration_magnitude_mss': vibration_magnitude_A, # <--- COLUNA ATUALIZADA
    'days_to_failure': days_to_failure_A,
    'failure_mode': failure_mode_A
})

df_motor_B = pd.DataFrame({
    'timestamp': timestamps,
    'device_id': device_id_B,
    'temperature_c': temp_B,
    'current_amps': current_B,
    'vibration_magnitude_mss': vibration_magnitude_B, # <--- COLUNA ATUALIZADA
    'days_to_failure': days_to_failure_B,
    'failure_mode': failure_mode_B
})

# Concatenar os DataFrames
df_simulated = pd.concat([df_motor_A, df_motor_B]).reset_index(drop=True)

# Arredondar para 2 casas decimais e garantir valores positivos
df_simulated['temperature_c'] = df_simulated['temperature_c'].round(2)
df_simulated['current_amps'] = df_simulated['current_amps'].round(2)
df_simulated['vibration_magnitude_mss'] = df_simulated['vibration_magnitude_mss'].round(2) # <--- COLUNA ATUALIZADA
df_simulated['days_to_failure'] = df_simulated['days_to_failure'].round(2)

print("Novo dataset simulado e corrigido gerado com sucesso!")
print("\n--- Início do Dataset (Motor A) ---")
print(df_simulated.head())
print("\n--- Final do Dataset (Motor B) ---")
print(df_simulated.tail())
print(f"\nTotal de registros: {len(df_simulated)}")

# Salvar para uso posterior
df_simulated.to_csv('simulated_sensor_data_corrected_vibration.csv', index=False)
print("\nDataset salvo como 'simulated_sensor_data_corrected_vibration.csv'")

Novo dataset simulado e corrigido gerado com sucesso!

--- Início do Dataset (Motor A) ---
            timestamp device_id  temperature_c  current_amps  \
0 2025-01-01 00:00:00   motor_A          30.99          1.59   
1 2025-01-01 00:10:00   motor_A          29.72          1.69   
2 2025-01-01 00:20:00   motor_A          31.30          1.36   
3 2025-01-01 00:30:00   motor_A          33.05          1.56   
4 2025-01-01 00:40:00   motor_A          29.53          1.43   

   vibration_magnitude_mss  days_to_failure failure_mode  
0                     9.77             -1.0       normal  
1                     9.79             -1.0       normal  
2                     9.76             -1.0       normal  
3                     9.78             -1.0       normal  
4                     9.71             -1.0       normal  

--- Final do Dataset (Motor B) ---
              timestamp device_id  temperature_c  current_amps  \
995 2025-01-04 10:30:00   motor_B          40.40          4.72   
99

### Importar módulos

### Configurações iniciais da simulação

### Parâmetros de dados normais (média e desvio padrão para ruído)

### Parâmetros de falha (taxa de aumento por dia)

### Geração dos dados